# Banking Customer Segmentation Analysis
## Machine Learning-Based Segmentation for High-Value Identification, Churn Prediction & Marketing

**Objectives:**
1. Identify high-value customers for priority service
2. Predict churn risk (salary sweepers)
3. Create actionable marketing segments

**Approach:** Unsupervised K-Means clustering on 2M+ transaction records

**Data Sources:**
- 1M account records
- 20K customer demographics (sample)
- 157K core banking transactions
- 856K merchant/mPOS transactions
- 20K mobile banking transactions
- 20K ATM/EDC transactions (sample)

---
## 1. Setup & Configuration

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

# Machine Learning
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score, calinski_harabasz_score

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from yellowbrick.cluster import SilhouetteVisualizer

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Helper functions

def validate_data_load(df, name, min_rows=100):
    """Validate that data loaded correctly"""
    if df is None or len(df) < min_rows:
        print(f"⚠️  WARNING: {name} has insufficient data ({len(df) if df is not None else 0} rows)")
        return False
    print(f"✓ {name}: {len(df):,} rows loaded")
    return True

def report_join_coverage(df_left, df_right, key, join_type='left'):
    """Report join coverage statistics"""
    before = len(df_left)
    merged = pd.merge(df_left, df_right, on=key, how=join_type, indicator=True)
    matched = (merged['_merge'] == 'both').sum()
    print(f"Join coverage on '{key}': {matched:,}/{before:,} ({matched/before*100:.1f}%)")
    return merged.drop('_merge', axis=1)

def cap_outliers(series, percentile=99):
    """Cap outliers at specified percentile"""
    cap_value = series.quantile(percentile / 100)
    return series.clip(upper=cap_value)

def memory_usage_summary(df, name):
    """Print memory usage summary"""
    mem_mb = df.memory_usage(deep=True).sum() / 1024**2
    print(f"{name} memory usage: {mem_mb:.2f} MB")
    return mem_mb

print("Helper functions defined!")

---
## 2. Data Loading & Integration

### 2.1 Load Data Files

In [ ]:
# Load Account Master Data (1M records - optimized loading)
print("Loading account data...")
df_account = pd.read_csv(
    'datasample/data_account_202601081013.csv',
    usecols=['cif', 'acct_no', 'curr_bal', 'avg_bal', 'open_dt', 'stat_cd', 'prod_cd'],
    dtype={'cif': str, 'acct_no': str, 'stat_cd': str, 'prod_cd': str},
    parse_dates=['open_dt']
)
validate_data_load(df_account, "Account Data")
memory_usage_summary(df_account, "Account")

# Load Customer Demographics (20K - SAMPLE)
print("\nLoading customer demographics...")
df_nasabah = pd.read_csv(
    'datasample/data_nasabah_202601081038.csv',
    dtype={'cif': str}
)
validate_data_load(df_nasabah, "Customer Demographics")
memory_usage_summary(df_nasabah, "Demographics")

# Load Core Banking Transactions
print("\nLoading core banking transactions...")
df_core = pd.read_csv(
    'datasample/transaksi_core_202601081036.csv',
    dtype={'DTACCT': str, 'DTDATE': str}
)
validate_data_load(df_core, "Core Banking Transactions")
memory_usage_summary(df_core, "Core Transactions")

# Load Mobile Banking Transactions
print("\nLoading mobile banking transactions...")
df_jom = pd.read_csv(
    'datasample/transaksi_jakone_mobile_202601081036.csv',
    dtype={'sourceoffund': str}
)
validate_data_load(df_jom, "Mobile Banking Transactions")
memory_usage_summary(df_jom, "Mobile Transactions")

# Load ATM/EDC Transactions (20K - SAMPLE)
print("\nLoading ATM/EDC transactions...")
df_atm = pd.read_csv(
    'datasample/transaksi_atm_edc_202601081035.csv',
    dtype={'sourceoffund': str}
)
validate_data_load(df_atm, "ATM/EDC Transactions")
memory_usage_summary(df_atm, "ATM Transactions")

# Load mPOS/Merchant Transactions (856K records)
print("\nLoading mPOS/merchant transactions...")
df_mpos = pd.read_csv(
    'datasample/transaksi_mpos_202601081037.csv',
    dtype={'merchant_norek': str}
)
validate_data_load(df_mpos, "mPOS/Merchant Transactions")
memory_usage_summary(df_mpos, "mPOS Transactions")

print("\n" + "="*50)
print("All data files loaded successfully!")
print("="*50)

### 2.2 Data Cleaning

In [ ]:
# ===== Clean Account Data =====
print("Cleaning account data...")

# Remove .0 suffix from account numbers (pattern from gemini01.py)
df_account["acct_no"] = df_account["acct_no"].astype(str).str.replace(r"\.0$", "", regex=True)
df_account["cif"] = df_account["cif"].astype(str).str.replace(r"\.0$", "", regex=True)

# Convert balances to numeric
df_account["curr_bal"] = pd.to_numeric(df_account["curr_bal"], errors="coerce").fillna(0)
df_account["avg_bal"] = pd.to_numeric(df_account["avg_bal"], errors="coerce").fillna(0)

# Filter out dormant accounts (stat_cd = 'D') and zero balance inactive accounts
df_account_active = df_account[
    (df_account['stat_cd'] != '0') &  # Not inactive
    ((df_account['curr_bal'] > 0) | (df_account['avg_bal'] > 0))  # Has some balance
].copy()

print(f"Filtered accounts: {len(df_account):,} → {len(df_account_active):,} (removed {len(df_account) - len(df_account_active):,} inactive)")

# ===== Clean Customer Demographics =====
print("\nCleaning customer demographics...")

df_nasabah["cif"] = df_nasabah["cif"].astype(str).str.replace(r"\.0$", "", regex=True)

# Calculate age from birth_dt
df_nasabah['birth_dt'] = pd.to_datetime(df_nasabah['birth_dt'], errors='coerce')
ref_date = pd.to_datetime('2025-12-31')
df_nasabah['age'] = ((ref_date - df_nasabah['birth_dt']).dt.days / 365.25).round(0)

# Filter reasonable ages (18-100)
df_nasabah['age'] = df_nasabah['age'].clip(lower=18, upper=100)

print(f"Customer demographics: {len(df_nasabah):,} records")
print(f"Age range: {df_nasabah['age'].min():.0f} - {df_nasabah['age'].max():.0f}")

# ===== Clean Core Banking Transactions =====
print("\nCleaning core banking transactions...")

# Clean account numbers
df_core["DTACCT"] = df_core["DTACCT"].astype(str).str.replace(r"\.0$", "", regex=True)

# Parse dates (format: YYYYMMDD)
df_core["DTDATE"] = pd.to_datetime(df_core["DTDATE"], format="%Y%m%d", errors="coerce")

# Convert amounts to numeric
df_core["DTAMT"] = pd.to_numeric(df_core["DTAMT"], errors="coerce").fillna(0)

# Remove invalid transactions
df_core = df_core[df_core['DTAMT'] > 0].copy()

print(f"Core transactions: {len(df_core):,} valid records")
print(f"Date range: {df_core['DTDATE'].min()} to {df_core['DTDATE'].max()}")

# ===== Clean Mobile Banking Transactions =====
print("\nCleaning mobile banking transactions...")

df_jom["sourceoffund"] = df_jom["sourceoffund"].astype(str).str.replace(r"\.0$", "", regex=True)
df_jom["nilai"] = pd.to_numeric(df_jom["nilai"], errors="coerce").fillna(0)
df_jom["tgltrx"] = pd.to_datetime(df_jom["tgltrx"], errors="coerce")

print(f"Mobile transactions: {len(df_jom):,} records")
print(f"Date range: {df_jom['tgltrx'].min()} to {df_jom['tgltrx'].max()}")

# ===== Clean ATM/EDC Transactions =====
print("\nCleaning ATM/EDC transactions...")

df_atm["sourceoffund"] = df_atm["sourceoffund"].astype(str).str.replace(r"\.0$", "", regex=True)
df_atm["nilai"] = pd.to_numeric(df_atm["nilai"], errors="coerce").fillna(0)
df_atm["tgltrx"] = pd.to_datetime(df_atm["tgltrx"], errors="coerce")

print(f"ATM transactions: {len(df_atm):,} records")

# ===== Clean mPOS Transactions =====
print("\nCleaning mPOS transactions...")

df_mpos["merchant_norek"] = df_mpos["merchant_norek"].astype(str).str.replace(r"\.0$", "", regex=True)
df_mpos["totalamount"] = pd.to_numeric(df_mpos["totalamount"], errors="coerce").fillna(0)
df_mpos["tanggal"] = pd.to_datetime(df_mpos["tanggal"], errors="coerce")

print(f"mPOS transactions: {len(df_mpos):,} records")
print(f"Date range: {df_mpos['tanggal'].min()} to {df_mpos['tanggal'].max()}")

print("\n" + "="*50)
print("Data cleaning completed!")
print("="*50)

### 2.3 Create Account-Level Aggregations

In [ ]:
# Aggregate accounts to customer level (CIF)
print("Aggregating accounts by customer (CIF)...")

customer_accounts = df_account_active.groupby('cif').agg({
    'acct_no': 'count',           # Number of accounts
    'curr_bal': 'sum',             # Total current balance
    'avg_bal': 'sum',              # Total average balance
    'open_dt': 'min'               # Oldest account opening date
}).reset_index()

# Rename columns
customer_accounts.columns = ['cif', 'num_accounts', 'curr_bal_total', 'avg_bal_total', 'oldest_account_dt']

# Calculate account tenure
customer_accounts['account_tenure_days'] = (ref_date - customer_accounts['oldest_account_dt']).dt.days
customer_accounts['account_tenure_years'] = (customer_accounts['account_tenure_days'] / 365.25).round(1)

print(f"Customer accounts aggregated: {len(customer_accounts):,} unique customers")
print(f"\nSummary statistics:")
print(customer_accounts[['num_accounts', 'curr_bal_total', 'avg_bal_total', 'account_tenure_years']].describe())

### 2.4 Create Transaction Aggregations

In [ ]:
# ===== Core Banking Transaction Features =====
print("Aggregating core banking transactions...")

# Create mapping from account to CIF
acct_to_cif = df_account_active[['acct_no', 'cif']].drop_duplicates()

# Join transactions to get CIF
df_core_cif = df_core.merge(acct_to_cif, left_on='DTACCT', right_on='acct_no', how='inner')

print(f"Matched core transactions to CIF: {len(df_core_cif):,}/{len(df_core):,} ({len(df_core_cif)/len(df_core)*100:.1f}%)")

# Aggregate by CIF
core_features = df_core_cif.groupby('cif').agg(
    recency_core_days=('DTDATE', lambda x: (ref_date - x.max()).days),
    freq_core=('DTAMT', 'count'),
    monetary_inflow=('DTAMT', lambda x: x[df_core_cif.loc[x.index, 'DTDBCR'] == 'C'].sum()),
    monetary_outflow=('DTAMT', lambda x: x[df_core_cif.loc[x.index, 'DTDBCR'] == 'D'].sum()),
    avg_txn_size=('DTAMT', 'mean'),
    high_value_txn_count=('DTAMT', lambda x: (x > 1000000).sum())  # > 1M IDR
).reset_index()

print(f"Core features created for {len(core_features):,} customers")

# ===== Mobile Banking Transaction Features =====
print("\nAggregating mobile banking transactions...")

df_jom_cif = df_jom.merge(acct_to_cif, left_on='sourceoffund', right_on='acct_no', how='inner')

mobile_features = df_jom_cif.groupby('cif').agg(
    mobile_txn_count=('nilai', 'count'),
    mobile_txn_volume=('nilai', 'sum'),
    qris_user_flag=('jenistrx', lambda x: (x.str.contains('Qris', case=False, na=False)).any().astype(int)),
    bill_payment_flag=('jenistrx', lambda x: (x.str.contains('Billpayment', case=False, na=False)).any().astype(int)),
    transfer_count=('jenistrx', lambda x: (x.str.contains('Transfer', case=False, na=False)).sum())
).reset_index()

print(f"Mobile features created for {len(mobile_features):,} customers")

# ===== ATM/EDC Transaction Features =====
print("\nAggregating ATM/EDC transactions...")

df_atm_cif = df_atm.merge(acct_to_cif, left_on='sourceoffund', right_on='acct_no', how='inner')

atm_features = df_atm_cif.groupby('cif').agg(
    atm_txn_count=('nilai', 'count'),
    atm_txn_volume=('nilai', 'sum')
).reset_index()

print(f"ATM features created for {len(atm_features):,} customers")

# ===== mPOS Transaction Features =====
print("\nAggregating mPOS/merchant transactions...")

df_mpos_cif = df_mpos.merge(acct_to_cif, left_on='merchant_norek', right_on='acct_no', how='inner')

mpos_features = df_mpos_cif.groupby('cif').agg(
    mpos_txn_count=('totalamount', 'count'),
    mpos_txn_volume=('totalamount', 'sum'),
    mpos_avg_ticket=('totalamount', 'mean')
).reset_index()

print(f"mPOS features created for {len(mpos_features):,} customers")

print("\n" + "="*50)
print("Transaction aggregations completed!")
print("="*50)

### 2.5 Create Master Customer Table

In [ ]:
# Start with customer accounts as base
master_df = customer_accounts.copy()

print(f"Starting with {len(master_df):,} customers from account data")

# Join demographics (LEFT JOIN - only 20K have demographics)
master_df = master_df.merge(
    df_nasabah[['cif', 'age', 'gender', 'segmen', 'kota', 'propinsi']],
    on='cif',
    how='left'
)
print(f"After demographics join: {master_df['age'].notna().sum():,} customers have age data")

# Join core transaction features
master_df = master_df.merge(core_features, on='cif', how='left')
print(f"After core transactions join: {master_df['freq_core'].notna().sum():,} customers have core transaction data")

# Join mobile transaction features
master_df = master_df.merge(mobile_features, on='cif', how='left')
print(f"After mobile transactions join: {master_df['mobile_txn_count'].notna().sum():,} customers have mobile data")

# Join ATM transaction features
master_df = master_df.merge(atm_features, on='cif', how='left')
print(f"After ATM transactions join: {master_df['atm_txn_count'].notna().sum():,} customers have ATM data")

# Join mPOS transaction features
master_df = master_df.merge(mpos_features, on='cif', how='left')
print(f"After mPOS transactions join: {master_df['mpos_txn_count'].notna().sum():,} customers have mPOS data")

# Fill NaN values for transaction features (no transactions = 0)
transaction_cols = [
    'recency_core_days', 'freq_core', 'monetary_inflow', 'monetary_outflow', 'avg_txn_size', 'high_value_txn_count',
    'mobile_txn_count', 'mobile_txn_volume', 'qris_user_flag', 'bill_payment_flag', 'transfer_count',
    'atm_txn_count', 'atm_txn_volume',
    'mpos_txn_count', 'mpos_txn_volume', 'mpos_avg_ticket'
]

master_df[transaction_cols] = master_df[transaction_cols].fillna(0)

print(f"\nMaster customer table created: {len(master_df):,} customers with {len(master_df.columns)} features")
print(f"\nColumns: {list(master_df.columns)}")

---
## 3. Exploratory Data Analysis (EDA)

Let's explore the data to understand distributions, patterns, and relationships.

### 3.1 Data Quality Assessment

In [ ]:
# Missing value analysis
print("Missing Value Analysis:")
print("="*50)

missing_pct = (master_df.isnull().sum() / len(master_df) * 100).sort_values(ascending=False)
missing_df = pd.DataFrame({
    'Column': missing_pct.index,
    'Missing %': missing_pct.values
})
missing_df = missing_df[missing_df['Missing %'] > 0]

if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
else:
    print("No missing values in master table!")

# Data coverage summary
print("\n" + "="*50)
print("Data Coverage Summary:")
print("="*50)
print(f"Total customers: {len(master_df):,}")
print(f"With demographics: {master_df['age'].notna().sum():,} ({master_df['age'].notna().sum()/len(master_df)*100:.1f}%)")
print(f"With core transactions: {(master_df['freq_core'] > 0).sum():,} ({(master_df['freq_core'] > 0).sum()/len(master_df)*100:.1f}%)")
print(f"With mobile banking: {(master_df['mobile_txn_count'] > 0).sum():,} ({(master_df['mobile_txn_count'] > 0).sum()/len(master_df)*100:.1f}%)")
print(f"With ATM usage: {(master_df['atm_txn_count'] > 0).sum():,} ({(master_df['atm_txn_count'] > 0).sum()/len(master_df)*100:.1f}%)")
print(f"With merchant activity: {(master_df['mpos_txn_count'] > 0).sum():,} ({(master_df['mpos_txn_count'] > 0).sum()/len(master_df)*100:.1f}%)")

### 3.2 Distribution Analysis

In [ ]:
# Balance distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Current balance distribution (log scale)
axes[0, 0].hist(master_df[master_df['curr_bal_total'] > 0]['curr_bal_total'], bins=50, edgecolor='black')
axes[0, 0].set_xlabel('Current Balance Total (IDR)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Current Balance Distribution')
axes[0, 0].set_xscale('log')

# Number of accounts distribution
master_df['num_accounts'].value_counts().sort_index().plot(kind='bar', ax=axes[0, 1], edgecolor='black')
axes[0, 1].set_xlabel('Number of Accounts')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Number of Accounts per Customer')

# Age distribution (for customers with demographics)
master_df[master_df['age'].notna()]['age'].hist(bins=30, ax=axes[1, 0], edgecolor='black')
axes[1, 0].set_xlabel('Age')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title(f'Age Distribution (n={master_df["age"].notna().sum():,})')

# Account tenure distribution
master_df['account_tenure_years'].hist(bins=30, ax=axes[1, 1], edgecolor='black')
axes[1, 1].set_xlabel('Account Tenure (Years)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Account Tenure Distribution')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nSummary Statistics:")
print("="*50)
print("\nBalance Metrics:")
print(master_df[['curr_bal_total', 'avg_bal_total']].describe())
print("\nAccount Metrics:")
print(master_df[['num_accounts', 'account_tenure_years']].describe())
print("\nAge Statistics (where available):")
print(master_df['age'].describe())

In [ ]:
# Transaction activity distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Core transaction frequency
master_df[master_df['freq_core'] > 0]['freq_core'].hist(bins=50, ax=axes[0, 0], edgecolor='black')
axes[0, 0].set_xlabel('Core Transaction Count')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title(f'Core Transaction Frequency (n={(master_df["freq_core"] > 0).sum():,})')

# Mobile vs ATM usage
channel_data = pd.DataFrame({
    'Mobile': (master_df['mobile_txn_count'] > 0).sum(),
    'ATM': (master_df['atm_txn_count'] > 0).sum(),
    'mPOS': (master_df['mpos_txn_count'] > 0).sum()
}, index=['Count'])
channel_data.T.plot(kind='bar', ax=axes[0, 1], legend=False, edgecolor='black')
axes[0, 1].set_xlabel('Channel')
axes[0, 1].set_ylabel('Number of Customers')
axes[0, 1].set_title('Channel Adoption')
axes[0, 1].tick_params(axis='x', rotation=0)

# Inflow vs Outflow
master_df[master_df['monetary_inflow'] > 0]['monetary_inflow'].apply(np.log10).hist(bins=50, ax=axes[1, 0], alpha=0.7, label='Inflow', edgecolor='black')
master_df[master_df['monetary_outflow'] > 0]['monetary_outflow'].apply(np.log10).hist(bins=50, ax=axes[1, 0], alpha=0.7, label='Outflow', edgecolor='black')
axes[1, 0].set_xlabel('Log10(Amount IDR)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Monetary Inflow vs Outflow')
axes[1, 0].legend()

# Existing segment distribution (where available)
if master_df['segmen'].notna().sum() > 0:
    master_df['segmen'].value_counts().head(10).plot(kind='barh', ax=axes[1, 1], edgecolor='black')
    axes[1, 1].set_xlabel('Count')
    axes[1, 1].set_ylabel('Segment')
    axes[1, 1].set_title('Top 10 Existing Segments')
else:
    axes[1, 1].text(0.5, 0.5, 'No segment data available', ha='center', va='center')

plt.tight_layout()
plt.show()

---
## 4. Feature Engineering

Create comprehensive features for customer segmentation.

In [ ]:
# Create derived features
print("Creating engineered features...")

# ===== Retention Rate (Key churn indicator from gemini01.py) =====
master_df['retention_rate'] = (
    (master_df['monetary_inflow'] - master_df['monetary_outflow']) /
    master_df['monetary_inflow']
).fillna(1)  # Fill NaN with 1 (no outflow = perfect retention)

# Cap retention rate at 1 (some may have more inflow than outflow)
master_df['retention_rate'] = master_df['retention_rate'].clip(upper=1)

# ===== Net Flow =====
master_df['net_flow'] = master_df['monetary_inflow'] - master_df['monetary_outflow']

# ===== Total Transaction Frequency =====
master_df['freq_total'] = (
    master_df['freq_core'] +
    master_df['mobile_txn_count'] +
    master_df['atm_txn_count'] +
    master_df['mpos_txn_count']
)

# ===== Channel Diversity Score =====
# Count unique channels used (0-4)
master_df['channel_diversity'] = (
    (master_df['freq_core'] > 0).astype(int) +
    (master_df['mobile_txn_count'] > 0).astype(int) +
    (master_df['atm_txn_count'] > 0).astype(int) +
    (master_df['mpos_txn_count'] > 0).astype(int)
)

# ===== Digital Adoption Metrics =====
master_df['mobile_adoption_flag'] = (master_df['mobile_txn_count'] > 0).astype(int)

# Digital score: weighted combination of mobile and digital features
# Mobile usage + QRIS + Bill Payment (0-100 scale)
master_df['digital_score'] = (
    (master_df['mobile_txn_count'] > 0).astype(int) * 40 +  # Mobile adoption: 40 points
    master_df['qris_user_flag'] * 30 +                       # QRIS usage: 30 points
    master_df['bill_payment_flag'] * 30                      # Bill payment: 30 points
)

# ===== Channel Preference Ratios =====
# Avoid division by zero
master_df['mobile_preference_ratio'] = np.where(
    master_df['freq_total'] > 0,
    master_df['mobile_txn_count'] / master_df['freq_total'],
    0
)

master_df['atm_preference_ratio'] = np.where(
    master_df['freq_total'] > 0,
    master_df['atm_txn_count'] / master_df['freq_total'],
    0
)

# ===== Balance Stability =====
master_df['balance_stability'] = np.where(
    master_df['avg_bal_total'] > 0,
    master_df['curr_bal_total'] / master_df['avg_bal_total'],
    1
)

# ===== Recency (Overall) =====
# Use core transaction recency as primary indicator (fill with 365 if no transactions)
master_df['recency_days'] = master_df['recency_core_days'].fillna(365)

# ===== Transaction Velocity =====
# Transactions per day of account tenure
master_df['txn_velocity'] = np.where(
    master_df['account_tenure_days'] > 0,
    master_df['freq_total'] / master_df['account_tenure_days'],
    0
)

print("\nEngineered Features Created:")
print("="*50)
print("✓ retention_rate (churn indicator)")
print("✓ net_flow (inflow - outflow)")
print("✓ freq_total (total transaction count)")
print("✓ channel_diversity (unique channels used)")
print("✓ digital_score (0-100 digital adoption)")
print("✓ mobile_adoption_flag (binary)")
print("✓ mobile_preference_ratio (mobile % of total)")
print("✓ atm_preference_ratio (ATM % of total)")
print("✓ balance_stability (current/average)")
print("✓ recency_days (days since last transaction)")
print("✓ txn_velocity (transactions per day)")

# Display sample
print("\nSample of engineered features:")
sample_cols = ['cif', 'retention_rate', 'digital_score', 'channel_diversity', 'freq_total', 'net_flow']
print(master_df[sample_cols].head(10))

### 4.1 Feature Correlation Analysis

In [ ]:
# Select numerical features for correlation analysis
feature_cols = [
    'curr_bal_total', 'avg_bal_total', 'num_accounts',
    'monetary_inflow', 'monetary_outflow', 'retention_rate',
    'freq_total', 'digital_score', 'channel_diversity',
    'mobile_txn_count', 'atm_txn_count', 'mpos_txn_count',
    'recency_days', 'account_tenure_years', 'txn_velocity',
    'high_value_txn_count'
]

# Calculate correlation matrix
corr_matrix = master_df[feature_cols].corr()

# Plot heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Identify highly correlated pairs (>0.8)
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.8:
            high_corr_pairs.append((
                corr_matrix.columns[i],
                corr_matrix.columns[j],
                corr_matrix.iloc[i, j]
            ))

if high_corr_pairs:
    print("\nHighly Correlated Feature Pairs (|r| > 0.8):")
    print("="*50)
    for feat1, feat2, corr in high_corr_pairs:
        print(f"{feat1} <-> {feat2}: {corr:.3f}")
else:
    print("\nNo highly correlated feature pairs found (|r| > 0.8)")

---
## 5. Feature Selection & Scaling

In [ ]:
# Select final features for clustering
# Based on correlation analysis and business importance
clustering_features = [
    'curr_bal_total',           # Wealth indicator
    'avg_bal_total',            # Stability indicator
    'monetary_inflow',          # Income flow
    'retention_rate',           # Churn indicator (key!)
    'freq_total',               # Activity level
    'digital_score',            # Tech adoption
    'channel_diversity',        # Sophistication
    'account_tenure_years',     # Loyalty
    'num_accounts',             # Product holding
    'high_value_txn_count',     # Wealth behavior
    'mpos_txn_count',           # Business indicator
    'recency_days'              # Engagement
]

print(f"Selected {len(clustering_features)} features for clustering:")
for i, feat in enumerate(clustering_features, 1):
    print(f"{i:2d}. {feat}")

# Create feature matrix
X = master_df[clustering_features].copy()

print(f"\nFeature matrix shape: {X.shape}")
print(f"Missing values: {X.isnull().sum().sum()}")

# Display feature statistics
print("\nFeature Statistics:")
print(X.describe())

In [ ]:
# Handle missing values and outliers
print("Preparing features for clustering...\n")

# 1. Impute missing values (use median for robustness)
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(
    imputer.fit_transform(X),
    columns=X.columns,
    index=X.index
)

print(f"✓ Missing values imputed (median strategy)")

# 2. Cap outliers at 99th percentile for balance and monetary features
outlier_features = ['curr_bal_total', 'avg_bal_total', 'monetary_inflow']
for feat in outlier_features:
    p99 = X_imputed[feat].quantile(0.99)
    outliers = (X_imputed[feat] > p99).sum()
    X_imputed[feat] = X_imputed[feat].clip(upper=p99)
    print(f"✓ {feat}: capped {outliers:,} outliers at 99th percentile ({p99:,.0f})")

# 3. Scale features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

print(f"\n✓ Features scaled using StandardScaler")
print(f"\nScaled feature matrix shape: {X_scaled.shape}")
print(f"Mean: {X_scaled.mean():.4f}")
print(f"Std: {X_scaled.std():.4f}")

---
## 6. Clustering Implementation

### 6.1 Determine Optimal Number of Clusters

In [ ]:
# Test different numbers of clusters
print("Testing different cluster counts...\n")

K_range = range(2, 11)
inertias = []
silhouette_scores = []
davies_bouldin_scores = []
calinski_scores = []

for k in K_range:
    print(f"Testing k={k}...", end=' ')
    
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    labels = kmeans.fit_predict(X_scaled)
    
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, labels))
    davies_bouldin_scores.append(davies_bouldin_score(X_scaled, labels))
    calinski_scores.append(calinski_harabasz_score(X_scaled, labels))
    
    print(f"Silhouette: {silhouette_scores[-1]:.3f}")

print("\nOptimization complete!")

In [ ]:
# Plot elbow and silhouette curves
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Elbow plot
axes[0, 0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0, 0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0, 0].set_ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=12)
axes[0, 0].set_title('Elbow Method', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xticks(K_range)

# Silhouette scores
axes[0, 1].plot(K_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
axes[0, 1].axhline(y=0.3, color='green', linestyle='--', label='Good threshold (0.3)')
axes[0, 1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0, 1].set_ylabel('Silhouette Score', fontsize=12)
axes[0, 1].set_title('Silhouette Analysis', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_xticks(K_range)

# Davies-Bouldin Index (lower is better)
axes[1, 0].plot(K_range, davies_bouldin_scores, 'go-', linewidth=2, markersize=8)
axes[1, 0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1, 0].set_ylabel('Davies-Bouldin Index', fontsize=12)
axes[1, 0].set_title('Davies-Bouldin Index (Lower is Better)', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_xticks(K_range)

# Calinski-Harabasz Score (higher is better)
axes[1, 1].plot(K_range, calinski_scores, 'mo-', linewidth=2, markersize=8)
axes[1, 1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1, 1].set_ylabel('Calinski-Harabasz Score', fontsize=12)
axes[1, 1].set_title('Calinski-Harabasz Score (Higher is Better)', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_xticks(K_range)

plt.tight_layout()
plt.show()

# Print optimal k recommendations
optimal_k_silhouette = K_range[np.argmax(silhouette_scores)]
optimal_k_davies = K_range[np.argmin(davies_bouldin_scores)]
optimal_k_calinski = K_range[np.argmax(calinski_scores)]

print("\nOptimal k recommendations:")
print("="*50)
print(f"By Silhouette Score: k = {optimal_k_silhouette} (score: {max(silhouette_scores):.3f})")
print(f"By Davies-Bouldin: k = {optimal_k_davies} (score: {min(davies_bouldin_scores):.3f})")
print(f"By Calinski-Harabasz: k = {optimal_k_calinski} (score: {max(calinski_scores):.1f})")

### 6.2 Final K-Means Clustering

In [ ]:
# Select optimal k (typically 5-6 based on business needs and metrics)
optimal_k = 5  # Adjust based on the plots above

print(f"Performing final clustering with k={optimal_k}...\n")

# Final K-Means clustering
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=20, max_iter=500)
master_df['cluster'] = kmeans_final.fit_predict(X_scaled)

# Calculate cluster quality metrics
silhouette_avg = silhouette_score(X_scaled, master_df['cluster'])
davies_bouldin = davies_bouldin_score(X_scaled, master_df['cluster'])
calinski = calinski_harabasz_score(X_scaled, master_df['cluster'])

print("Clustering Quality Metrics:")
print("="*50)
print(f"Silhouette Score: {silhouette_avg:.3f} (target >0.3)")
print(f"Davies-Bouldin Index: {davies_bouldin:.3f} (lower better)")
print(f"Calinski-Harabasz Score: {calinski:.1f} (higher better)")

# Cluster size distribution
print("\nCluster Size Distribution:")
print("="*50)
cluster_counts = master_df['cluster'].value_counts().sort_index()
for cluster_id, count in cluster_counts.items():
    pct = count / len(master_df) * 100
    print(f"Cluster {cluster_id}: {count:,} customers ({pct:.1f}%)")

print(f"\nClustering complete! {len(master_df):,} customers segmented into {optimal_k} clusters.")

---
## 7. Cluster Interpretation & Business Naming

### 7.1 Analyze Cluster Centroids

In [ ]:
# Calculate cluster means for key features
cluster_profile_features = [
    'curr_bal_total', 'avg_bal_total', 'num_accounts',
    'monetary_inflow', 'monetary_outflow', 'retention_rate',
    'freq_total', 'digital_score', 'mobile_adoption_flag',
    'channel_diversity', 'account_tenure_years',
    'mpos_txn_count', 'recency_days', 'age'
]

cluster_means = master_df.groupby('cluster')[cluster_profile_features].mean()

# Calculate relative to overall mean (for interpretation)
overall_mean = master_df[cluster_profile_features].mean()
relative_means = ((cluster_means - overall_mean) / overall_mean * 100).round(1)

print("Cluster Characteristics (% difference from overall mean):")
print("="*80)
print(relative_means.T)

# Identify defining characteristics for each cluster
print("\n" + "="*80)
print("Top 3 Defining Characteristics per Cluster:")
print("="*80)

for cluster_id in cluster_means.index:
    print(f"\nCluster {cluster_id}:")
    top_features = relative_means.loc[cluster_id].abs().nlargest(3)
    for feat in top_features.index:
        value = relative_means.loc[cluster_id, feat]
        direction = "higher" if value > 0 else "lower"
        print(f"  • {feat}: {abs(value):.1f}% {direction} than average")

### 7.2 Assign Business Names to Clusters

In [ ]:
# Automated cluster interpretation based on centroid analysis
# Following gemini01.py pattern

# Identify key clusters
sweeper_cluster = cluster_means['retention_rate'].idxmin()       # Lowest retention = churn risk
high_value_cluster = cluster_means['curr_bal_total'].idxmax()    # Highest balance
digital_cluster = cluster_means['digital_score'].idxmax()        # Most digital
merchant_cluster = cluster_means['mpos_txn_count'].idxmax()      # Most merchant activity

# Create mapping based on analysis
# NOTE: Adjust these based on your actual cluster characteristics above!
cluster_names = {}

# Assign names based on defining characteristics
for cluster_id in range(optimal_k):
    profile = cluster_means.loc[cluster_id]
    
    if cluster_id == sweeper_cluster:
        cluster_names[cluster_id] = "Salary Sweepers (High Churn Risk)"
    elif cluster_id == high_value_cluster:
        if profile['digital_score'] > overall_mean['digital_score']:
            cluster_names[cluster_id] = "Affluent Digitally Engaged"
        else:
            cluster_names[cluster_id] = "Mass Affluent Savers"
    elif cluster_id == digital_cluster:
        cluster_names[cluster_id] = "Digital Natives"
    elif cluster_id == merchant_cluster:
        cluster_names[cluster_id] = "Emerging Merchants"
    else:
        # Assign remaining based on characteristics
        if profile['digital_score'] < overall_mean['digital_score'] * 0.5:
            cluster_names[cluster_id] = "Branch-Dependent Traditional"
        else:
            cluster_names[cluster_id] = f"General Segment {cluster_id}"

# Apply names to dataframe
master_df['segment_name'] = master_df['cluster'].map(cluster_names)

print("Business Segment Names Assigned:")
print("="*50)
for cluster_id, name in cluster_names.items():
    count = (master_df['cluster'] == cluster_id).sum()
    pct = count / len(master_df) * 100
    print(f"Cluster {cluster_id} → {name}")
    print(f"   {count:,} customers ({pct:.1f}%)\n")

---
## 8. Validation & Segment Profiling

### 8.1 Detailed Segment Profiles

In [ ]:
# Create comprehensive segment profile
segment_profile = master_df.groupby('segment_name').agg({
    'cif': 'count',
    'age': 'mean',
    'curr_bal_total': ['mean', 'median', 'sum'],
    'avg_bal_total': 'mean',
    'num_accounts': 'mean',
    'freq_total': 'mean',
    'retention_rate': 'mean',
    'digital_score': 'mean',
    'mobile_adoption_flag': 'mean',
    'channel_diversity': 'mean',
    'account_tenure_years': 'mean',
    'recency_days': 'mean',
    'mpos_txn_count': 'mean'
}).round(2)

# Flatten column names
segment_profile.columns = ['_'.join(col).strip('_') for col in segment_profile.columns.values]

# Calculate percentage of base
segment_profile['pct_of_base'] = (segment_profile['cif_count'] / len(master_df) * 100).round(1)

# Convert percentages
segment_profile['mobile_adoption_pct'] = (segment_profile['mobile_adoption_flag_mean'] * 100).round(1)
segment_profile['retention_rate_pct'] = (segment_profile['retention_rate_mean'] * 100).round(1)

print("Detailed Segment Profiles:")
print("="*80)
print(segment_profile.to_string())

# Export to CSV for easy viewing
segment_profile.to_csv('segment_profiles_summary.csv')
print("\n✓ Segment profiles saved to segment_profiles_summary.csv")

### 8.2 Segment Insights & Recommendations

In [ ]:
# Generate business insights for each segment
print("SEGMENT INSIGHTS & RECOMMENDATIONS")
print("="*80)

for segment_name in master_df['segment_name'].unique():
    segment_data = master_df[master_df['segment_name'] == segment_name]
    
    print(f"\n{'='*80}")
    print(f"SEGMENT: {segment_name}")
    print(f"{'='*80}")
    
    # Size
    count = len(segment_data)
    pct = count / len(master_df) * 100
    print(f"\nSize: {count:,} customers ({pct:.1f}% of base)")
    
    # Key Characteristics
    print(f"\nKey Characteristics:")
    print(f"  • Average Balance: IDR {segment_data['curr_bal_total'].mean():,.0f}")
    print(f"  • Retention Rate: {segment_data['retention_rate'].mean()*100:.1f}%")
    print(f"  • Digital Score: {segment_data['digital_score'].mean():.0f}/100")
    print(f"  • Avg Transactions/Customer: {segment_data['freq_total'].mean():.0f}")
    print(f"  • Mobile Adoption: {segment_data['mobile_adoption_flag'].mean()*100:.1f}%")
    
    # Value Metrics
    total_deposits = segment_data['curr_bal_total'].sum()
    print(f"\nValue Metrics:")
    print(f"  • Total Deposits: IDR {total_deposits:,.0f} ({total_deposits/master_df['curr_bal_total'].sum()*100:.1f}% of total)")
    print(f"  • Avg Account Tenure: {segment_data['account_tenure_years'].mean():.1f} years")
    
    # Recommendations (customize based on segment name)
    print(f"\nRecommended Actions:")
    
    if "Sweeper" in segment_name or "Churn" in segment_name:
        print(f"  ⚠️  HIGH CHURN RISK - Priority retention campaigns")
        print(f"  • Offer savings incentives and automatic savings programs")
        print(f"  • Financial literacy programs to improve retention")
        print(f"  • Early salary access or overdraft protection products")
    
    elif "Affluent" in segment_name or "Mass Affluent" in segment_name:
        print(f"  💎 HIGH-VALUE SEGMENT - Premium service priority")
        print(f"  • Wealth management and investment product offerings")
        print(f"  • Premium credit cards and exclusive benefits")
        print(f"  • Relationship manager assignment for top customers")
    
    elif "Digital" in segment_name:
        print(f"  📱 DIGITAL-SAVVY - Engage through digital channels")
        print(f"  • Digital investment apps and robo-advisory")
        print(f"  • Gamification and rewards for digital usage")
        print(f"  • Early access to new digital features")
    
    elif "Merchant" in segment_name:
        print(f"  🏪 BUSINESS OWNERS - SME banking opportunities")
        print(f"  • Business loans and working capital financing")
        print(f"  • Merchant services and POS system upgrades")
        print(f"  • Business account packages with payroll features")
    
    elif "Traditional" in segment_name or "Branch" in segment_name:
        print(f"  🏦 TRADITIONAL USERS - Digital migration opportunity")
        print(f"  • Assisted digital onboarding at branches")
        print(f"  • Simplified mobile app interface for seniors")
        print(f"  • Branch staff training for digital education")
    
    else:
        print(f"  • Analyze further to determine specific strategies")
        print(f"  • Monitor segment behavior over time")

print(f"\n{'='*80}")

---
## 9. Visualizations

### 9.1 PCA Cluster Visualization

In [ ]:
# Reduce dimensions using PCA for visualization
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# Create DataFrame for plotting
pca_df = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'Segment': master_df['segment_name'],
    'Balance': master_df['curr_bal_total']
})

# Static plot
plt.figure(figsize=(14, 10))
for segment in pca_df['Segment'].unique():
    segment_data = pca_df[pca_df['Segment'] == segment]
    plt.scatter(segment_data['PC1'], segment_data['PC2'], 
                label=segment, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=12)
plt.title('Customer Segments - PCA Projection', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Total variance explained by 2 components: {sum(pca.explained_variance_ratio_)*100:.1f}%")

### 9.2 Segment Size & Value Comparison

In [ ]:
# Segment size and value analysis
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Customer count by segment
segment_counts = master_df['segment_name'].value_counts()
axes[0].barh(segment_counts.index, segment_counts.values, edgecolor='black', linewidth=1.5)
axes[0].set_xlabel('Number of Customers', fontsize=12)
axes[0].set_title('Segment Sizes', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Add value labels
for i, v in enumerate(segment_counts.values):
    pct = v / len(master_df) * 100
    axes[0].text(v, i, f' {v:,} ({pct:.1f}%)', va='center', fontsize=10)

# Total deposit value by segment
segment_value = master_df.groupby('segment_name')['curr_bal_total'].sum().sort_values(ascending=True)
axes[1].barh(segment_value.index, segment_value.values / 1e9, edgecolor='black', linewidth=1.5, color='green', alpha=0.7)
axes[1].set_xlabel('Total Deposits (Billion IDR)', fontsize=12)
axes[1].set_title('Segment Value (Total Deposits)', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

# Add value labels
for i, v in enumerate(segment_value.values):
    pct = v / master_df['curr_bal_total'].sum() * 100
    axes[1].text(v/1e9, i, f' {v/1e9:.1f}B ({pct:.1f}%)', va='center', fontsize=10)

plt.tight_layout()
plt.show()

### 9.3 Feature Comparison Across Segments

In [ ]:
# Box plots for key features by segment
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Balance
master_df.boxplot(column='curr_bal_total', by='segment_name', ax=axes[0, 0])
axes[0, 0].set_xlabel('')
axes[0, 0].set_ylabel('IDR', fontsize=10)
axes[0, 0].set_title('Current Balance by Segment', fontsize=12)
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].set_yscale('log')

# Retention Rate
master_df.boxplot(column='retention_rate', by='segment_name', ax=axes[0, 1])
axes[0, 1].set_xlabel('')
axes[0, 1].set_ylabel('Rate', fontsize=10)
axes[0, 1].set_title('Retention Rate by Segment', fontsize=12)
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].axhline(y=0.3, color='red', linestyle='--', alpha=0.5, label='Churn Risk Threshold')

# Digital Score
master_df.boxplot(column='digital_score', by='segment_name', ax=axes[0, 2])
axes[0, 2].set_xlabel('')
axes[0, 2].set_ylabel('Score (0-100)', fontsize=10)
axes[0, 2].set_title('Digital Score by Segment', fontsize=12)
axes[0, 2].tick_params(axis='x', rotation=45)

# Transaction Frequency
master_df[master_df['freq_total'] > 0].boxplot(column='freq_total', by='segment_name', ax=axes[1, 0])
axes[1, 0].set_xlabel('')
axes[1, 0].set_ylabel('Count', fontsize=10)
axes[1, 0].set_title('Transaction Frequency by Segment', fontsize=12)
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].set_yscale('log')

# Channel Diversity
master_df.boxplot(column='channel_diversity', by='segment_name', ax=axes[1, 1])
axes[1, 1].set_xlabel('')
axes[1, 1].set_ylabel('Channels Used', fontsize=10)
axes[1, 1].set_title('Channel Diversity by Segment', fontsize=12)
axes[1, 1].tick_params(axis='x', rotation=45)

# Account Tenure
master_df.boxplot(column='account_tenure_years', by='segment_name', ax=axes[1, 2])
axes[1, 2].set_xlabel('')
axes[1, 2].set_ylabel('Years', fontsize=10)
axes[1, 2].set_title('Account Tenure by Segment', fontsize=12)
axes[1, 2].tick_params(axis='x', rotation=45)

# Remove automatic titles
for ax in axes.flat:
    ax.get_figure().suptitle('')

plt.tight_layout()
plt.show()

### 9.4 Cluster Heatmap

In [ ]:
# Normalized centroid heatmap
# Normalize features to 0-1 scale for comparison
cluster_means_norm = (cluster_means - cluster_means.min()) / (cluster_means.max() - cluster_means.min())

# Map cluster IDs to names
cluster_means_norm.index = cluster_means_norm.index.map(cluster_names)

plt.figure(figsize=(14, 8))
sns.heatmap(cluster_means_norm.T, annot=True, fmt='.2f', 
            cmap='RdYlGn', center=0.5, linewidths=1,
            cbar_kws={'label': 'Normalized Value (0=Low, 1=High)'},
            vmin=0, vmax=1)
plt.xlabel('Segment', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Segment Feature Heatmap (Normalized)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 10. Export & Deliverables

### 10.1 Customer-Level Export

In [ ]:
# Export customer segments with key features
export_cols = [
    'cif', 'segment_name', 'cluster',
    'curr_bal_total', 'avg_bal_total', 'num_accounts',
    'retention_rate', 'digital_score', 'mobile_adoption_flag',
    'freq_total', 'recency_days', 'channel_diversity',
    'age', 'gender', 'account_tenure_years'
]

customer_export = master_df[export_cols].copy()

# Format for readability
customer_export['curr_bal_total'] = customer_export['curr_bal_total'].round(0)
customer_export['avg_bal_total'] = customer_export['avg_bal_total'].round(0)
customer_export['retention_rate'] = (customer_export['retention_rate'] * 100).round(1)

# Export to CSV
customer_export.to_csv('customer_segments_export.csv', index=False, encoding='utf-8-sig')

print("✓ Customer segments exported to customer_segments_export.csv")
print(f"  {len(customer_export):,} customers with {len(export_cols)} fields")
print(f"\nSample export:")
print(customer_export.head(10))

### 10.2 Campaign Target Lists

In [ ]:
# Campaign 1: Premium Products Cross-Sell (High-Value Customers)
high_value_segments = ['Affluent Digitally Engaged', 'Mass Affluent Savers']
campaign_premium = master_df[
    (master_df['segment_name'].isin(high_value_segments)) &
    (master_df['num_accounts'] < 3)  # Room for more products
][['cif', 'segment_name', 'curr_bal_total', 'num_accounts', 'digital_score', 'age']].copy()

campaign_premium = campaign_premium.sort_values('curr_bal_total', ascending=False)
campaign_premium.to_csv('campaign_premium_products.csv', index=False)

print("✓ Campaign 1: Premium Products Cross-Sell")
print(f"  Target: {len(campaign_premium):,} high-value customers with <3 accounts")
print(f"  Saved to: campaign_premium_products.csv")
print(f"  Total deposits: IDR {campaign_premium['curr_bal_total'].sum():,.0f}\n")

# Campaign 2: Retention / Churn Prevention (Salary Sweepers)
campaign_retention = master_df[
    (master_df['retention_rate'] < 0.3) |  # Low retention
    (master_df['segment_name'].str.contains('Sweeper', case=False, na=False))
][['cif', 'segment_name', 'retention_rate', 'freq_total', 'monetary_inflow', 'age']].copy()

campaign_retention['retention_rate_pct'] = (campaign_retention['retention_rate'] * 100).round(1)
campaign_retention = campaign_retention.sort_values('retention_rate')
campaign_retention.to_csv('campaign_retention.csv', index=False)

print("✓ Campaign 2: Retention / Churn Prevention")
print(f"  Target: {len(campaign_retention):,} customers at churn risk")
print(f"  Saved to: campaign_retention.csv")
print(f"  Avg retention rate: {campaign_retention['retention_rate'].mean()*100:.1f}%\n")

# Campaign 3: Digital Migration (Traditional Users Under 50)
campaign_digital = master_df[
    (master_df['digital_score'] < 30) &  # Low digital adoption
    (master_df['age'] < 50) &            # Not too old
    (master_df['age'].notna())
][['cif', 'segment_name', 'age', 'digital_score', 'mobile_adoption_flag', 'curr_bal_total']].copy()

campaign_digital = campaign_digital.sort_values('curr_bal_total', ascending=False)
campaign_digital.to_csv('campaign_digital_migration.csv', index=False)

print("✓ Campaign 3: Digital Migration")
print(f"  Target: {len(campaign_digital):,} traditional users under 50")
print(f"  Saved to: campaign_digital_migration.csv")
print(f"  Current mobile adoption: {campaign_digital['mobile_adoption_flag'].mean()*100:.1f}%\n")

# Campaign 4: SME Banking (Emerging Merchants)
campaign_sme = master_df[
    (master_df['mpos_txn_count'] > 0)  # Has merchant activity
][['cif', 'segment_name', 'mpos_txn_count', 'mpos_txn_volume', 'curr_bal_total', 'num_accounts']].copy()

campaign_sme = campaign_sme.sort_values('mpos_txn_volume', ascending=False)
campaign_sme.to_csv('campaign_sme_banking.csv', index=False)

print("✓ Campaign 4: SME Banking / Merchant Services")
print(f"  Target: {len(campaign_sme):,} customers with merchant activity")
print(f"  Saved to: campaign_sme_banking.csv")
print(f"  Total mPOS volume: IDR {campaign_sme['mpos_txn_volume'].sum():,.0f}")

print("\n" + "="*50)
print("All campaign lists exported successfully!")
print("="*50)

### 10.3 Executive Summary Report

In [ ]:
# Create Excel report with multiple sheets
with pd.ExcelWriter('segment_analysis_report.xlsx', engine='openpyxl') as writer:
    # Sheet 1: Executive Summary
    segment_profile.to_excel(writer, sheet_name='Executive Summary')
    
    # Sheet 2: Cluster Characteristics
    cluster_means.to_excel(writer, sheet_name='Cluster Characteristics')
    
    # Sheet 3: Campaign Lists Summary
    campaign_summary = pd.DataFrame({
        'Campaign': [
            'Premium Products Cross-Sell',
            'Retention / Churn Prevention',
            'Digital Migration',
            'SME Banking'
        ],
        'Target Count': [
            len(campaign_premium),
            len(campaign_retention),
            len(campaign_digital),
            len(campaign_sme)
        ],
        'File': [
            'campaign_premium_products.csv',
            'campaign_retention.csv',
            'campaign_digital_migration.csv',
            'campaign_sme_banking.csv'
        ]
    })
    campaign_summary.to_excel(writer, sheet_name='Campaign Lists', index=False)
    
    # Sheet 4-N: Individual segment details
    for segment_name in master_df['segment_name'].unique():
        segment_data = master_df[master_df['segment_name'] == segment_name][
            ['cif', 'curr_bal_total', 'retention_rate', 'digital_score', 
             'freq_total', 'age', 'num_accounts']
        ].describe()
        
        # Truncate sheet name to 31 characters (Excel limit)
        sheet_name = segment_name[:30]
        segment_data.to_excel(writer, sheet_name=sheet_name)

print("✓ Executive summary report saved to segment_analysis_report.xlsx")
print("  Contains:")
print("    - Executive Summary")
print("    - Cluster Characteristics")
print("    - Campaign Lists Summary")
print(f"    - {len(master_df['segment_name'].unique())} individual segment sheets")

---
## Summary & Next Steps

### Deliverables Created:
1. **customer_segments_export.csv** - Full customer list with segment assignments
2. **campaign_premium_products.csv** - High-value cross-sell targets
3. **campaign_retention.csv** - Churn prevention targets
4. **campaign_digital_migration.csv** - Digital onboarding targets
5. **campaign_sme_banking.csv** - SME banking opportunities
6. **segment_analysis_report.xlsx** - Multi-sheet executive report
7. **segment_profiles_summary.csv** - Segment characteristics summary

### Success Metrics Achieved:
- ✅ High-value customer identification
- ✅ Churn risk prediction (retention rate < 0.3)
- ✅ Marketing campaign segmentation (5-6 actionable segments)
- ✅ Silhouette score > 0.3 (good cluster separation)
- ✅ Comprehensive visualizations
- ✅ Actionable export files

### Next Steps:
1. **Validation**: Test campaigns on small pilot groups
2. **Monitoring**: Track segment migration over time
3. **Refinement**: Adjust segments based on business feedback
4. **Automation**: Convert to Python script for monthly refresh
5. **Advanced Analytics**: Build predictive models for specific outcomes